In [4]:
!pip install sagemaker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 919.4/919.4 kB 2.3 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Using cached attrs-23.1.0-py3-none-any.whl (61 kB)
  Using cached cloudpickle-2.2.1-py3-none-any.whl (25 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Obtaining dependency information for protobuf<5.0,>=3.12 from https://files.pythonhosted.org/packages/e6/db/7b2edc72807d45d72f9db42f3eb86ddaf37f9e55d923159b1dbfc9d835bc/protobuf-4.25.1-cp37-abi3-macosx_10_9_universal2.whl.metadata
  Using cached smdebug_rulesconfig-1.0.1-py2.py3-none-any.whl (20 kB)
  Obtaining dependency information for importlib-metadata<7.0,>=1.4.0 from https://files.pythonhosted.org/packages/cc/37/db7ba97e676af155f5fcb1a35466f446eadc9104e25b83366e8088c9c926/importlib_metadata-6.8.0-py3-none-any.whl.metadata
  Obtaining dependency information for pathos from https://files.pythonhosted.org/packages/d8/08/ac94fa6f9eefe32963b8a54e573dab0dbc0d3df24fd34924bd9ce7eab7c4/patho

In [1]:
import boto3

# Create an IAM client
iam = boto3.client('iam')

# List roles
response = iam.list_roles()

# If you want to filter for SageMaker roles, you can do it in Python
sagemaker_roles = [role for role in response['Roles'] if 'SageMaker' in role['RoleName']]

# `sagemaker_roles` now contains a list of SageMaker roles

In [2]:
sagemaker_roles[0]['RoleName']

'AmazonSageMaker-ExecutionRole-20231030T210397'

In [3]:
import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='AmazonSageMaker-ExecutionRole-20231030T210397')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/ravi.tej/Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/ravi.tej/Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/ravi.tej/Library/Application Support/sagemaker/config.yaml


Couldn't call 'get_role' to get Role ARN from role name ravi_tej to get Role path.


sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/ravi.tej/Library/Application Support/sagemaker/config.yaml
sagemaker role arn: arn:aws:iam::005418323977:role/service-role/AmazonSageMaker-ExecutionRole-20231030T210397
sagemaker bucket: sagemaker-ap-south-1-005418323977
sagemaker session region: ap-south-1


In [4]:
import pandas as pd
import numpy as np
import json

In [107]:
system_prompt =  '''
You are the chief editor for a leading Indian financial and business news website. You evaluate critical attributes of articles to gate keep content quality. For many attributes, you will first provide a brief analysis of 15 to 30 words, followed by assessment.

1. analysis_of_financial_or_business_news (short text) : <analyse if article pertains to finance/business or not. government policies directly impacting indian corporations or investors are ok, but not if aren't>
2. financial_or_business_news (True or False) : <True or False based on previous attribute>
3. analysis_of_relevant_for_india (short text) : <analyse if article is relevant for indians. for example international articles about 401k or small foreign companies won't be relevant for india. however changes to fed interest rates or nasdaq or important news of large multinational corporations will be relevant>
4. relevant_for_india (True or False) : <True or False based on previous attribute>
5. analysis_of_article_validity_duration (short text) : <analyse relevance duration. Be stingy: Stock fluctuations, 1 day; significant policy changes - few days or a week; educational content with references to any regulations is 30 unless there are none - in which case is timeless. International news in India has shorter lifespan. breaking news are usually not timeless; quarterly analysis or results are usually valid for a 3 days, yearly analysis or results for a weeks and a much longer one for a month.>
6. article_validity_duration (one of 1, 3, 7, 14, 30, -1) : <calculate number of days based on previous attribute. -1: timeless. 1: article is relevant only for that day. 3: for a couple of days. 7: for a week. 14: for a couple of weeks. 30: for a month>
7. analysis_of_popularity (short text) : <analyse likely popularity of article - if its for niche audience, moderate_popularity or should be part of breaking_news section, depending on number of people who will be impacted by the news and the scale of the event. foreign entities known in india but not very popular will be mostly niche or rarely moderately popular. articles targeted to very specific business or pratices will be niche. infotainment business and financial articles with some drama are likely to be more popular. articles with a list of rules without compelling story-telling will be for niche audience>
8. popularity (one of niche, moderately_popular, breaking_news) : <based on previous attribute>
9. analysis_of_article_type (short text) : <analyse if the article is majorly factual, is an opinion piece, analysis, educational or likely sponsored. factual articles pass on information on events. opinion pieces have inferences or predictions either from the author or from statements without data. analysis pieces have substantial data to justify their inferences or predictions. if an article is overly zealous on certain stock and seems like an ad, then it is sponsored>
10. article_type (one of fact, opinion, analysis, educational, sponsored) : <based on previous attribute>
11. analysis_of_article_sentiment (short_text): <analyse if the sentiment of the article is bullish, bearish or NA. balanced is NA>
12. article_sentiment (one of bull, bear, na): <based on previous attribute>
13. headline_suggestion (short text) : <Write a headline based on the content of the article>
14. first_attempt_summary (text of 60 words) : <Generate concise, entity-dense summary. The summary should become highly dense but easily understood without the Article. Don't keep the summary too short, but limit it to no more than 60 words>
15. improved_summary (text of 60 words): <Identify contents of the article which are missing from the previous summary but are important part of the article>
16. final_summary (text of 60 words): <The finalised summary which is a mixture of first_attempt_summary and improved_summary. This summary should be very dense and also all the important information of the article and yet concise. By reading this in most cases the users need not read the article>
17. top_categories (5 semi colon seperated categories): <Hierarchy of 5 categories or keywords. Start with a generic category and make it progressively specific. Select only 5 and seperate them by (;). Don't use either single or double quotes at any cost to avoid json.loads() failure>

your response should be a json structure with all the 17 above keys without missing any key. It is very important that the response is directly readable with json.loads(). no preamble or postamble. respond in the exact following structure:

{
"analysis_of_financial_or_business_news": "",
"financial_or_business_news": "",
"analysis_of_relevant_for_india": "",
"relevant_for_india": "",
"analysis_of_article_validity_duration": "",
"article_validity_duration": "",
"analysis_of_popularity": "",
"popularity": "",
"analysis_of_article_type": "",
"article_type": "",
"analysis_of_article_sentiment": "",
"article_sentiment": "",
"headline_suggestion": "",
"first_attempt_summary": "",
"improved_summary": "",
"final_summary": "",
"top_categories": ""
}

'''

In [82]:
endpoint_name = 'huggingface-pytorch-tgi-inference-2023-12-01-07-00-06-142'

In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('teknium/OpenHermes-2.5-Mistral-7B', use_fast=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
import re

In [7]:
def calculate_tokens(text, encoder):  # Placeholder for your actual token calculation function
    # Your implementation will go here.
    return len(encoder.encode(text))  # Example: counting characters as tokens

def truncate_text_to_token_limit(text,encoder, token_limit):
    # First, check if the whole text is under the token limit
    if calculate_tokens(text, encoder) <= token_limit:
        return text  # The entire text is within the limit

    def is_under_limit(index):
        # Use the provided function to calculate tokens for the substring
        return calculate_tokens(text[:index], encoder) <= token_limit

    left, right = 0, len(text)
    valid_limit = 0  # This will hold the index of the last valid token position

    # Binary search to find the token limit
    while left <= right:
        mid = (left + right) // 2  # Find the midpoint
        if is_under_limit(mid):
            # If the midpoint is under the limit, store it as a valid limit
            valid_limit = mid
            left = mid + 1  # Move the left boundary to the right
        else:
            right = mid - 1  # Move the right boundary to the left

    # Find the last space before the valid_limit to ensure we're at a word boundary
    space_index = text.rfind(' ', 0, valid_limit)
    if space_index == -1:
        # If there's no space, we've hit the start of the text
        return text[:valid_limit]  # Return up to the valid limit even if mid-word

    # Return the text up to the last word within the token limit
    return text[:space_index]

In [108]:
def format_article_for_prompt(content):
    truncated_content = truncate_text_to_token_limit(text=content, encoder=tokenizer, token_limit=ARTICLE_TOKEN_LIMIT)
    messages = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": f"|article_start|\n {truncated_content}\n|article_end|\n"}]
    context_prompt = tokenizer.decode(tokenizer.apply_chat_template(messages, add_generation_prompt=True))
    context_prompt = re.sub(r'\n+','\n',context_prompt)
    return context_prompt

In [54]:
message_template = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": f"|article_start|\n {''}\n|article_end|\n"}]

OUTPUT_TOKEN_LIMIT = 1024 # set based on the distribution of completion tokens from gpt4
INSTRUCTION_TOKENS = len(tokenizer.apply_chat_template(message_template, add_generation_prompt=True))
BUFFER_TOKENS = 10
ARTICLE_TOKEN_LIMIT = 4096 - OUTPUT_TOKEN_LIMIT - INSTRUCTION_TOKENS - BUFFER_TOKENS
ARTICLE_TOKEN_LIMIT

1714

In [22]:
parent_folder = '/Users/ravi.tej/Desktop/ML/Recommendations/arcane/'
from hydra import compose, initialize
import os

import xml.etree.ElementTree as ET

tree = ET.parse('../../../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import sys
sys.path.append('/Users/ravi.tej/Desktop/ML/Recommendations/arcane/')

from src._utils import load_bertopic_model_from_hf

/Users/ravi.tej/anaconda3/envs/recoenv/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/ravi.tej/anaconda3/envs/recoenv/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/ravi.tej/anaconda3/envs/recoenv/lib/python3.11/site-pac

In [23]:
from src.articles.ArticleService import ArticleService

In [25]:
art = ArticleService._get_article_json_from_s3_and_api('6555c7f84b13023f9348f4b0')

In [26]:
full_content = art['title'] + art['cleaned_text']

In [27]:
full_content

'Stocks making the biggest moves midday: Rivian, Orchard Therapeutics, Lamb Weston and moreCheck out the companies making headlines in midday trading. Rivian Automotive — Rivian Automotive shares tanked 19% after the electric vehicle maker announced plans to raise $1.5 billion in convertible notes and offered preliminary third-quarter revenue guidance roughly in line with Wall Street\'s expectations. Rivian said it anticipates revenue to range between $1.29 billion and $1.33 billion, versus the $1.3 billion forecast by analysts polled by LSEG, formerly known as Refinitiv. Exxon Mobil — Shares slid more than 2.3% in midday trading following a further decline in oil prices on the back of an uncertain demand outlook and macroeconomic future. Clorox — Shares dropped 7.7% on Thursday, one day after the product maker offered worse fiscal first-quarter guidance than analysts polled by FactSet expected. The company said a cyberattack overshadowed benefits from better pricing, cost reduction an

In [28]:
smr = sess.boto_session.client("sagemaker-runtime")

In [55]:
parameters = {
    "do_sample": True,
    "top_p": 0.9,
    "temperature": 1.0,
    "max_new_tokens": OUTPUT_TOKEN_LIMIT,
    "repetition_penalty": 1.03,
    "stop": ["###", "</s>", tokenizer.eos_token],
}

In [117]:
OUTPUT_TOKEN_LIMIT

1024

In [56]:
import time
import re

In [57]:
request = {"inputs": format_article_for_prompt(full_content), "parameters": parameters, "stream": False}

In [58]:
start_time = time.time()
resp = smr.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(request),
    ContentType="application/json",
)
print(f'done in {time.time() - start_time}')

done in 29.084964990615845


In [59]:
k = resp['Body'].read()

In [60]:
json.loads(json.loads(k)[0]['generated_text'])

{'analysis_of_article_sentiment': 'The article shows diverse aspects of financial performance, thus without clear bullish or bearish sentiment.',
 'analysis_of_article_type': 'This article is majorly factual, providing data and facts about the stocks and their performance',
 'analysis_of_article_validity_duration': 'This is report on a Stock fluctuation which changes daily, therefore of high time sensitivity',
 'analysis_of_financial_or_business_news': 'Article discusses varied stock performances, starting from Auto to Chem well then to pharm and several others',
 'analysis_of_popularity': 'The article covers the stock movements of different companies, which would interest a broad range of readers',
 'analysis_of_relevant_for_india': 'It includes international companies like Rivian and IBM contributing to the account, thus is relevant for India',
 'article_sentiment': 'na',
 'article_type': 'fact',
 'article_validity_duration': 1,
 'final_summary': "Rivian's shares plummet 19% as it ra

In [62]:
len(tokenizer.encode(json.loads(k)[0]['generated_text']))

627

In [89]:
def get_attributes_from_llm(article_id):
    try:
        art = ArticleService._get_article_json_from_s3_and_api(article_id)
    except:
        print(article_id)
        return article_id, None
    full_content = art['title'] + art['cleaned_text']
    full_response = {}
    full_response['content'] = full_content
    full_input = format_article_for_prompt(full_content)
    # truncated_input = truncate_text_to_token_limit(full_input, encoder=tokenizer, token_limit=3072)
    request = {"inputs": full_input, "parameters": parameters, "stream": False}
    try:
        resp = smr.invoke_endpoint(
            EndpointName=endpoint_name,
            Body=json.dumps(request),
            ContentType="application/json",
            )
        k = resp['Body'].read()
        try:
            full_response['llm_attributes'] = json.loads(k)[0]['generated_text']
            return article_id, full_response
        except:
            return article_id, k
    except:
        return article_id, ''

In [68]:
from concurrent.futures import ThreadPoolExecutor, as_completed

### Testing

In [64]:
with open('../test_set_full_27_nov.json', 'r') as f:
    test_set = json.load(f)

In [65]:
test_article_ids = list(test_set.keys())

In [90]:
def get_attributes_for_article_ids(article_ids, max_retries=1, retry_count=0):
    start_time = time.time()
    responses = {}
    valid_attributes = {}
    invalid_response_ids = []

    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(get_attributes_from_llm, art_id) for art_id in article_ids]
        
        for future in as_completed(futures):
            art_id, response = future.result()
            if response is None:  # Timeout or other errors
                invalid_response_ids.append(art_id)
                continue

            responses[art_id] = response
            try:
                valid_attributes[art_id] = json.loads(response['llm_attributes'])
            except:
                invalid_response_ids.append(art_id)

    print(f'Attempt {retry_count + 1}: Processed {len(article_ids)} IDs in {time.time() - start_time} seconds')

    if invalid_response_ids and retry_count < max_retries:
        print(f'Retrying for {len(invalid_response_ids)} invalid IDs')
        valid_attributes_retry = get_attributes_for_article_ids(invalid_response_ids, max_retries, retry_count + 1)
        valid_attributes.update(valid_attributes_retry)

    return valid_attributes

In [109]:
va_new_4 = get_attributes_for_article_ids(test_article_ids[30:60])

6555cbf84b13023f934a382d
653168c51e5cc42b1b13b90b
Attempt 1: Processed 30 IDs in 529.7714197635651 seconds
Retrying for 8 invalid IDs
653168c51e5cc42b1b13b90b
6555cbf84b13023f934a382d
Attempt 2: Processed 8 IDs in 120.3303587436676 seconds


In [111]:
va_new_4.keys()

dict_keys(['6555cc3a4b13023f934a68c8', '6555b5d84b13023f9348c264', '651dd877a662d76276b7e743', '6555c49e4b13023f9348d93e', '65316f1d1e5cc42b1b13eff1', '6555b5044b13023f9348b8db', '651e14c4a662d76276b87d81', '6555c2e64b13023f9348d32b', '651df8e7a662d76276b83a76', '65316f9b1e5cc42b1b140b85', '653680241e5cc42b1b1440fc', '651e1036a662d76276b872e4', '6555c8144b13023f934905b9', '6555c0da4b13023f9348cd0d', '651de1bea662d76276b7ffb8', '6555c7c34b13023f9348e053', '6555b5a54b13023f9348be03', '651ddcd8a662d76276b7f2b4', '653168971e5cc42b1b13b0f9', '6555ca8b4b13023f934944d1', '6555cbb74b13023f934a0b16', '65316f4f1e5cc42b1b13f8c0', '65316f021e5cc42b1b13ec21', '651e1c4ea662d76276b88f01', '652fd3521e5cc42b1b13a524', '651dd2cea662d76276b7d91e', '6531692b1e5cc42b1b13d499'])

In [115]:
endpoint_name

'huggingface-pytorch-tgi-inference-2023-12-01-07-00-06-142'

In [114]:
['top_categories' in k for i, k in va_new_4.items()]

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [110]:
len(va_new_4)

27

In [98]:
va_new_3 = get_attributes_for_article_ids(test_article_ids[:30])

Attempt 1: Processed 30 IDs in 481.2182312011719 seconds
Retrying for 7 invalid IDs
Attempt 2: Processed 7 IDs in 97.00438594818115 seconds


In [101]:
len(va_new_3)

29

In [106]:
va_new_3['651de15fa662d76276b7fecf']

{'analysis_of_article_sentiment': 'The article keeps a neutral tone, talking about the process without presenting a bullish or bearish sentiment.',
 'analysis_of_article_type': 'This is a factual news piece providing information on an online system to expedite property registration',
 'analysis_of_article_validity_duration': 'The article is about a significant government policy change with long-term implications, but the initial implementation (and related progress and issues) will stay relevant for a few days',
 'analysis_of_financial_or_business_news': "Article discusses the government's initiative to fast-track property registration.",
 'analysis_of_popularity': 'This article discusses a government process change which could impact many Delhi citizens, making it of moderate popular interest',
 'analysis_of_relevant_for_india': "The article discusses the Indian government's government-e-Stores system, it is relevant to Indian population.",
 'article_sentiment': 'na',
 'article_type':

In [102]:
va_new_3.keys()

dict_keys(['65316faa1e5cc42b1b141024', '6555cc534b13023f934a7aa8', '6555c1124b13023f9348ce03', '651de15fa662d76276b7fecf', '652ebb961e5cc42b1b139924', '6555cb974b13023f9349f4ae', '651e1b59a662d76276b88cc8', '651e1f32a662d76276b895fe', '6555cc4c4b13023f934a75cf', '6555c5694b13023f9348dcc0', '6555cac44b13023f9349673f', '651debaaa662d76276b819f6', '651de04fa662d76276b7fbfb', '651e000ca662d76276b84c0d', '652ebbe61e5cc42b1b139c4f', '6555c3c24b13023f9348d582', '6531691c1e5cc42b1b13d0c9', '651def1aa662d76276b822fe', '6555c31b4b13023f9348d38e', '6555cbba4b13023f934a0d12', '651dd87da662d76276b7e750', '65316d2a1e5cc42b1b13e1fc', '652cdf9950af0e25a912312f', '651e1a84a662d76276b88aee', '6555cc194b13023f934a4fa1', '65316f961e5cc42b1b1409ef', '6555ccc84b13023f934ad130', '6555ccc94b13023f934ad203', '65316f5e1e5cc42b1b13fbde'])

In [93]:
va_new_2 = get_attributes_for_article_ids(test_article_ids)

6555cbf84b13023f934a382d
653168c51e5cc42b1b13b90b
65316f521e5cc42b1b13f95e
Attempt 1: Processed 100 IDs in 1541.0907452106476 seconds
Retrying for 100 invalid IDs
6555cbf84b13023f934a382d
653168c51e5cc42b1b13b90b
65316f521e5cc42b1b13f95e
Attempt 2: Processed 100 IDs in 1676.7273581027985 seconds


In [97]:
va_new_2

{}

In [71]:
va_new = get_attributes_for_article_ids(test_article_ids)

6555cbf84b13023f934a382d
653168c51e5cc42b1b13b90b
65316f521e5cc42b1b13f95e
Attempt 1: Processed 100 IDs in 1498.6548023223877 seconds
Retrying for 38 invalid IDs
653168c51e5cc42b1b13b90b6555cbf84b13023f934a382d

65316f521e5cc42b1b13f95e
Attempt 2: Processed 38 IDs in 738.4900138378143 seconds
Retrying for 17 invalid IDs
653168c51e5cc42b1b13b90b
6555cbf84b13023f934a382d
65316f521e5cc42b1b13f95e
Attempt 3: Processed 17 IDs in 285.04372811317444 seconds


In [72]:
va_new

{'6555c1124b13023f9348ce03': {'analysis_of_article_sentiment': 'The sentiment of the article is bearish regarding Russian influences in international politics and financial sanctions.',
  'analysis_of_article_type': 'The article gives out the facts regarding the sanctions from the U.S. towards Russian entities, including names of the entities, and the effects on the U.S. economy.',
  'analysis_of_article_validity_duration': 'The information presented is a sanctions decision by the U.S. on Russian entities which will remain a policy that is impactful for a considerable amount of time.',
  'analysis_of_financial_or_business_news': 'The article is related to financial sanctions imposed by the U.S. on Russian entities. It details the political impact and economic repercussions, especially on the energy sector.',
  'analysis_of_popularity': 'The article is about a prominent political issue with far-reaching consequences. it catches a wide audience, especially those interested in internation

In [76]:
[v for x, v in va_new.items() if not isinstance(v, dict)]

[]

In [77]:
endpoint_name = 'huggingface-pytorch-tgi-inference-2023-12-01-00-40-05-774'

In [78]:
va_old = get_attributes_for_article_ids(test_article_ids[:])

6555cbf84b13023f934a382d
653168c51e5cc42b1b13b90b
65316f521e5cc42b1b13f95e
Attempt 1: Processed 100 IDs in 2839.4906499385834 seconds
Retrying for 100 invalid IDs


KeyboardInterrupt: 

In [225]:
va['65367fe21e5cc42b1b143c7d']

{'analysis_is_financial_or_business_news': "The article discusses Reliance Industries' diversification into various sectors.",
 'is_financial_or_business_news': True,
 'analysis_of_relevant_for_india': 'The article is about Reliance, an Indian multinational corporation, and its growth in various sectors.',
 'relevant_for_india': True,
 'analysis_of_article_validity_duration': 'The article discusses long-term business strategies and growth of Reliance, making it timeless.',
 'article_validity_duration': -1,
 'analysis_of_popularity': 'Reliance Industries is a popular company in India, so the article will attract a wide audience.',
 'popularity': 'breaking_news',
 'analysis_of_article_type': "The article provides factual information about Reliance's business diversification.",
 'article_type': 'fact',
 'analysis_of_article_sentiment': "The article presents a bullish sentiment towards Reliance's growth and diversification.",
 'article_sentiment': 'bull',
 'headline_suggestion': "Reliance'